In [1]:
# Standard library imports
import os
from collections import Counter

# Third-party library imports
import torch
from torch import tensor
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import tiktoken
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util

# LangChain-specific imports
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore, Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings


c:\Users\dodsone\PycharmProjects\NLP\Projects\AgenticInformationRetrieval\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load .env file values 
load_dotenv()

# Access the environment variables
openai_api_key = os.getenv('OPENAI_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_qa = os.getenv('PINECONE_QA')
pc_context = os.getenv('PINECONE_CONTEXT')

In [3]:

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
autoModel = AutoModel.from_pretrained("bert-base-uncased")

In [4]:
# Function to encode a single string
def encode_text(text):
    # Tokenize the input text
    encoded_input = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Get the model output
    with torch.no_grad():
        output = autoModel(**encoded_input)
    
    # Use the mean pooling of token embeddings as the sentence embedding
    sentence_embedding = output.last_hidden_state.mean(dim=1)
    return sentence_embedding

In [5]:
# instantiate OpenAI client with API key
client = OpenAI(
    api_key=openai_api_key
)

# initializing Pinecone vector databases instance
qaSearch = Pinecone(
    index_name=pc_qa,
    embedding=OpenAIEmbeddings(openai_api_key=openai_api_key)
)

contextSearch = Pinecone(
    index_name=pc_context,
    embedding=OpenAIEmbeddings(openai_api_key=openai_api_key)
)

C:\Users\dodsone\AppData\Local\Temp\ipykernel_31528\1207440793.py:7: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 1.0.0. Use :class:`~PineconeVectorStore` instead.
  qaSearch = Pinecone(


In [6]:
# appends one/few shot examples to evaluation prompt
def few_shot(examples, evaluation_prompt):
   for example in examples:
       # separates example key and values
       split_ex = example.split(' - ')
       question = split_ex[0]
       label = split_ex[1]
    #    context_result = contextSearch.max_marginal_relevance_search(label, k=1, fetch_k=5)
    #    evaluation_prompt += '\n\nContext: \"' + context_result[0].page_content + "\"" + '\nQuestion: \"' + question + "\"" + '\n' + label
       evaluation_prompt += '\n\nQuestion: \"' + question + "\"" + '\n' + label

   evaluation_prompt += '\n\nAnswer the following question:'
   print(f'Finalized evaluation prompt... RAG complete!')

   return evaluation_prompt

In [7]:
# queries vector database for custom,
# with similar examples to user prompt
def rag(text, question):
    two_shots = []
    # queries Pinecone database
    print(f'Relevance Search...')
    qa_results = qaSearch.max_marginal_relevance_search(text, k=3, fetch_k=10)
    print(f'Finished querying!')
    for i in range(len(qa_results)):
        content = qa_results[i].page_content
        # prevents repetition which will cause errors within OpenAI
        two_shots.append(content)
        # two valid examples found
        if len(two_shots) == 2:
            break
    context_result = contextSearch.max_marginal_relevance_search(question, k=1, fetch_k=5)[0].page_content
    print(f'Context Length: {len(context_result)}')
    print(f'Context Value: {context_result}')
    text += f'\n\nContext: {context_result}'
    print(f'Appending shots...')
    return few_shot(two_shots, text)

In [8]:
# tests accuracy of chosen model against unique prompt and data
def evaluation(data, evaluation_prompt, model_name):

    test = pd.DataFrame(columns=['Question', 'Expected Answer', 'GPT Answer', 'Similarity'])
    for index, row in data.iterrows():
        # Extract the question
        question = row['Question']
        expected_answer = row['Answer']

        print(f"Starting Completion at Index: {index}")
        completion = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "system", 
                    "content": rag(evaluation_prompt, question)
                },
                {
                    "role": "user", 
                    "content": question
                }
            ]
        )
        print(f'Finished Completion {index}!')
        gpt_answer = completion.choices[0].message.content.strip()

        # append results
        test.loc[index] = [question, expected_answer, gpt_answer, None]

    return test

In [9]:
# retrieves data to test accuracy against model of choice
df_e = pd.read_csv('../../data/evaluation_data.csv')
df_c = pd.read_csv('../../data/context_data.csv')
df_c = df_c.drop(columns=['Unnamed: 0'])
df_e = df_e.drop(columns=['Unnamed: 0'])
df_c

,Passage
0,Uruguay official full name pron Eastern Republ...
1,bordered Brazil north Argentina across bank Ur...
2,Montevideo founded Spanish early 18th century ...
3,economy largely based agriculture making 10 GD...
4,According Transparency International Uruguay s...
...,...
3195,2007 duck Tallahassee Florida survived gunshot...
3196,rare genetic mutation sees ducks born four leg...
3197,Moche people ancient Peru worshipped nature Be...
3198,Angel Wing disease common ducks


In [10]:
# test chat bot with general model or fine-tuning model of choice
# *model* should include available OpenAI models for evaluation from link above
model = "gpt-3.5-turbo-0125"
evaluation_prompt = "You are a highly intelligent AI trained to provide detailed and accurate answers to user questions by leveraging a knowledge base. Below are examples of questions and their respective answers from the knowledge base. Keep the answers meaningful, but very short and concise"

In [11]:
# execute function for testing model against unique samples
result = evaluation(df_e, evaluation_prompt, model)

Starting Completion at Index: 0
Relevance Search...
Finished querying!
Context Length: 1150
Context Value: 279. Passage: Abraham Lincoln February 12 1809 â April 15 1865 sixteenth President United States serving March 4 1861 assassination outspoken opponent expansion slavery United States short autobiography written 1860 presidential campaign Lincoln would describe protest Illinois legislature one briefly defined position slavery question far goes reference antiexpansion sentiments expressed Doris Kearns Goodwin Team Rivals Political Genius Abraham Lincoln 2005 p 91 Holzer pg 232 Writing Cooper Union speech Holzer notes Cooper Union proved unique confluence political culture rhetorical opportunity technological innovation human genius brought Abraham Lincoln center stage American politics precisely right time place precisely right message slavery wrong ought confined areas already existed placed course ultimate extinction Lincoln Republican Party nomination 1860 elected president later

In [19]:
result

,Question,Expected Answer,GPT Answer,Similarity
0,Abraham Lincoln sixteenth President United States,yes,Abraham Lincoln was the sixteenth President of...,-1.0
1,Lincoln sign National Banking Act 1863,yes,Yes.,1.0
2,mother die pneumonia,,I'm sorry to hear that.,-1.0
3,many long Lincolns formal education,18 months,18 months,1.0
4,Lincoln begin political career,1832,1832,1.0
...,...,...,...,...
913,Wilson president American Political Science As...,Yes,Woodrow Wilson was the president of the Americ...,-1.0
914,cast ballot John Palmer presidential candidate...,Yes,What is the question you want me to answer?,1.0
915,Wilson spend 1914 beginning 1917 trying keep A...,Yes,To prevent America from entering into war with...,-1.0
916,Wilson staunch opponent antisemitism sympathet...,Yes,Yes.,1.0


In [20]:
# cleanse
result['Expected Answer'] = result['Expected Answer'].fillna("").astype(str)
result['GPT Answer'] = result['GPT Answer'].fillna("").astype(str)

In [21]:
# generate embeddings for consine similarity 
dataset_embeddings = torch.stack([encode_text(answer) for answer in result['Expected Answer'].tolist()])
gpt_embeddings = torch.stack([encode_text(answer) for answer in result['GPT Answer'].tolist()])

In [22]:
cosine_sim = torch.nn.functional.cosine_similarity(dataset_embeddings, gpt_embeddings)
print("Cosine Similarities:", cosine_sim)

Cosine Similarities: tensor([[-1.,  1.,  1.,  ..., -1., -1.,  1.],
        [ 1., -1., -1.,  ...,  1.,  1., -1.],
        [-1., -1.,  1.,  ..., -1.,  1., -1.],
        ...,
        [-1., -1.,  1.,  ..., -1.,  1.,  1.],
        [ 1., -1., -1.,  ...,  1.,  1., -1.],
        [ 1., -1., -1.,  ...,  1., -1., -1.]])


In [23]:
result['Similarity'] = cosine_sim
threshold = 0.8
correct = result[result['Similarity'] > threshold].shape[0]
total = result.shape[0]
accuracy = correct / total * 100

In [24]:
accuracy

63.725490196078425

In [25]:
# accuracy result 59.04139433551199
print(f"Accuracy: {accuracy * 1:.2f}%")

Accuracy: 63.73%
